In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input/breast-cancer-prediction-dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/breast-cancer-prediction-dataset/Breast_cancer_data.csv


In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from mlxtend.frequent_patterns import fpgrowth, apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder


In [4]:
df = pd.read_csv("/kaggle/input/breast-cancer-prediction-dataset/Breast_cancer_data.csv")
df.head(7)

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0
5,12.45,15.70,82.57,477.1,0.12780,0
6,18.25,19.98,119.60,1040.0,0.09463,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mean_radius      569 non-null    float64
 1   mean_texture     569 non-null    float64
 2   mean_perimeter   569 non-null    float64
 3   mean_area        569 non-null    float64
 4   mean_smoothness  569 non-null    float64
 5   diagnosis        569 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 26.8 KB


In [6]:

df.shape

(569, 6)

In [7]:

df.describe()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.627417
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.483918
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.000000
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.000000
50%,13.370000,18.840000,86.240000,551.100000,0.095870,1.000000
75%,15.780000,21.800000,104.100000,782.700000,0.105300,1.000000
max,28.110000,39.280000,188.500000,2501.000000,0.163400,1.000000


In [8]:
cols = df.columns
cols = cols.str.replace('catg_','')
cols = cols.str.replace('_',' ')
df.columns = cols
df.head(5)


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0


In [11]:


static_data = df.groupby('mean smoothness').sum().reset_index()

for col in static_data.columns[1:]:
    static_data[col] = static_data[col].apply(lambda x: 1 if x>0 else 0)
    
static_data.head(5)

,mean smoothness,mean radius,mean texture,mean perimeter,mean area,diagnosis
0,0.05263,1,1,1,1,1
1,0.06251,1,1,1,1,1
2,0.06429,1,1,1,1,1
3,0.06576,1,1,1,1,1
4,0.06613,1,1,1,1,1


In [12]:
patterns = fpgrowth(static_data.iloc[:,1:], min_support= 0.00002, use_colnames= True)

frequent_sets = patterns.sort_values('support', ascending= False)[:10]
frequent_sets.support = frequent_sets.support * len(static_data)
frequent_sets

/opt/conda/lib/python3.7/site-packages/mlxtend/frequent_patterns/fpcommon.py:115: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  DeprecationWarning,


,support,itemsets
0,474.0,(mean area)
9,474.0,"(mean texture, mean perimeter)"
25,474.0,"(mean area, mean radius, mean texture, mean pe..."
21,474.0,"(mean radius, mean texture, mean perimeter)"
18,474.0,"(mean area, mean radius, mean texture)"
16,474.0,"(mean area, mean radius, mean perimeter)"
1,474.0,(mean perimeter)
12,474.0,"(mean radius, mean texture)"
10,474.0,"(mean radius, mean perimeter)"
15,474.0,"(mean area, mean texture, mean perimeter)"


In [13]:
rules = association_rules(patterns, metric= 'confidence', min_threshold= 0.1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(mean area),(mean perimeter),1.000000,1.000000,1.000000,1.000000,1.0,0.0,inf
1,(mean perimeter),(mean area),1.000000,1.000000,1.000000,1.000000,1.0,0.0,inf
2,(mean area),(mean texture),1.000000,1.000000,1.000000,1.000000,1.0,0.0,inf
3,(mean texture),(mean area),1.000000,1.000000,1.000000,1.000000,1.0,0.0,inf
4,(mean area),(mean radius),1.000000,1.000000,1.000000,1.000000,1.0,0.0,inf
...,...,...,...,...,...,...,...,...,...
175,(mean texture),"(mean area, mean radius, diagnosis, mean perim...",1.000000,0.687764,0.687764,0.687764,1.0,0.0,1.0
176,(mean perimeter),"(diagnosis, mean area, mean radius, mean texture)",1.000000,0.687764,0.687764,0.687764,1.0,0.0,1.0
177,(mean radius),"(mean area, diagnosis, mean texture, mean peri...",1.000000,0.687764,0.687764,0.687764,1.0,0.0,1.0
178,(diagnosis),"(mean area, mean radius, mean texture, mean pe...",0.687764,1.000000,0.687764,1.000000,1.0,0.0,inf
